In [ ]:
#!pip install imutils

In [ ]:
#!pip install --upgrade protobuf

In [2]:
!pip install tensorflow-keras


ERROR: Could not find a version that satisfies the requirement tensorflow-keras (from versions: none)
ERROR: No matching distribution found for tensorflow-keras


In [1]:
# python detect_mask_video.py

# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import tensorflow as tf
from tensorflow import keras
import numpy as np
import argparse
import imutils
import time
import cv2
import os

ModuleNotFoundError: No module named 'tensorflow.keras'

In [ ]:
# to capture the image 
def detect_and_predict_mask(frame,faceNet,maskNet):
    # to get the dimension of the frame and to construct the blob
    (h,w)= frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame,1.0,(300,300),
                                 (104.0,177.0,123.0))
    
    #pass the blob through model and detect the face using faceNet
    faceNet.setInput(blob)
    detections = faceNet.forward()
    
    #initialize the list of face and the corresponding locatins and the list of predicitons from our face mask model
    
    faces =[]
    locs =[]
    preds =[]
    
    # loop over detections
    for i in range(0,detections.shape[2]):
        # extract the probability
        confidence = detections[0,0,i,2]
        
        #filter out weak detections by ensuring the confidence/probability value
        # (i.e) the confidence value is greater than the min conf value
        
        if confidence>args['confidence']:
            #compute the x and y coordinates of bounding box 
            
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startX,startY,endX,endY)=box.astype("int")
            
            #ensure the box falls within the face dim
            (startX,startY) =(max(0,startX),max(0,startY))
            (endX,endY) = (min(w-1,endX),min(h-1,endY))
            face = frame[startY:endY,startX:endX]

            # Debugging statements
            print(f"Detection {i}: confidence={confidence}")
            print(f"Bounding box: startX={startX}, startY={startY}, endX={endX}, endY={endY}")
            print(f"Face shape: {face.shape}")

            if face.size == 0:
                print("Empty face image detected, skipping...")
                continue
            
            #extract the face ROI, convert it to BGR to RGB channel ordering, resize it to 224x224, and preprocess it
            # face = frame[startY:endY,startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face,(224,224))
            face = img_to_array(face)
            face = preprocess_input(face)
            
            # add the face and bounding box to the respective list
            faces.append(face)
            locs.append((startX,startY,endX,endY))
            
    # to give a condition to predict atleast one face was detected
    if len(faces)>0:
# for faster inference we'll make batch predictions on *all*
# faces at the same time rather than one-by-one predictions
# in the above `for` loop
        faces = np.array(faces,dtype ='float32')
        preds = maskNet.predict(faces,batch_size = 32)
        
    return (locs,preds)
        


In [ ]:
# construct the argument parser and parse the arguments
# Construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-f", "--face", type=str, default="face_detector", help="path to face detector model directory")
ap.add_argument("-m", "--model", type=str, default="mask_detector.h5", help="path to trained face mask detector model")
ap.add_argument("-c", "--confidence", type=float, default=0.5, help="minimum probability to filter weak detections")
args = vars(ap.parse_args())

# Construct the absolute paths to the model files
project_root = os.path.abspath(os.path.dirname(r"E:\face mask detection\face_detector"))
face_detector_directory = os.path.join(project_root, 'face_detector')
prototxtPath = os.path.join(face_detector_directory, "deploy.prototxt")
weightsPath = os.path.join(face_detector_directory, "res10_300x300_ssd_iter_140000.caffemodel")

print(f"Prototxt path: {prototxtPath}")
print(f"Weights path: {weightsPath}")

if not os.path.exists(prototxtPath):
    print(f"Error: {prototxtPath} not found.")
if not os.path.exists(weightsPath):
    print(f"Error: {weightsPath} not found.")

faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)



In [ ]:
import keras
print(keras.__version__)


In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
# Load the face mask detector model from disk
print("[INFO] loading face mask detector model...")
maskNet = load_model(args["model"])


In [ ]:
# initialize the video stream and allow the camera sensor to warm up
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)



In [ ]:
# loop over the frames from the video stream
while True:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=400)

    # detect faces in the frame and determine if they are wearing a
    # face mask or not
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

    # loop over the detected face locations and their corresponding
    # locations
    for (box, pred) in zip(locs, preds):
        # unpack the bounding box and predictions
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred

        # determine the class label and color we'll use to draw
        # the bounding box and text
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        # include the probability in the label
        #label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        
        # display the label and bounding box rectangle on the output
        # frame
        if(label=="Mask"):    
            cv2.putText(frame,"Mask: You are allowed", (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
        elif(label=="No Mask"):
            lab="No Mask: You are not allowed"
            cv2.putText(frame, lab, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# do a bit of cleanup
cv2.destroyAllWindows()

vs.release()